In [10]:
# -*- coding: utf-8 -*-
"""
CreateNetworks.py

This algorithm aggregates the temporally resolved network data
default= 20min aggregation, by setting mins=1/3 we get 20sec resolution.

Output: Tensor A20 for day 1, tensor B20 for day 2 along with the the times as vectors.

@author: 
Yuki M. Asano
edited by:Jacqueline
"""
import numpy as np
def createnw(data,metadata,mins):
    tt=-1
    maxtime=int(np.ceil((data[-1,0] - data[0,0] )/(20*3*mins))) # 20min     
    numIndividuals=len(metadata[:, 0])
    startid=int(metadata[0][0])
    A= np.zeros((numIndividuals,numIndividuals, maxtime+1),dtype=np.int32)
    told=0  
    
    for row in range(len(data[:,0])):
        t=data[row,0]
        id1=int(np.argwhere(str(data[row,1])== metadata[:,0]))
        id2=int(np.argwhere(str(data[row,2])== metadata[:,0]))
        if (t>= (told+(20*3*mins))) and t!=told: 	#start new timeslot
            tt+=1
            told=t
        if id1>id2:					#fill lower triangular
            A[id1][id2][tt]+=1
        else:
            A[id2][id1][tt]+=1
    return A, range(tt)

In [11]:
import pandas as pd

df = pd.read_csv('primaryschool.csv', delimiter='\t', header=None)

data = np.array(df)
#data=np.loadtxt('primaryschool.csv', delimiter='\t')
firstday=data[0:60623,:]
secondday=data[60623:,:]
metadata=np.loadtxt('metadata_primaryschool.txt', delimiter='\t', dtype='S16')
metadata = metadata.astype(str)

no = len(metadata[:, 0])
# create 20min aggregated data
[A20,time] =createnw(firstday,metadata,1/3)
[B20,time2] =createnw(secondday,metadata,1/3)

# save data as numpy objects
np.save('day1.npy', A20)
np.save('day2.npy', B20)
np.save('numbers.npy',no)
np.save('times1.npy',time)
np.save('times2.npy',time2)